In [15]:
import numpy as np
import pandas as pd
from wikidataquery import WikidataQuery

## Let's consider the number of total academic conferences of Wikidata (P31/instance of='academic conference')

In [10]:
temp = WikidataQuery.queryWikiData(
    '''SELECT
    ?conferencesLabel
    ?seriesLabel
    ?short_name
    ?beginnings
    ?WikiCFP_identifier
    ?DBLP_identifier

    WHERE {
    ?conferences wdt:P31 wd:Q2020153.

    OPTIONAL {
    ?conferences wdt:P179 ?series.
    OPTIONAL { ?series wdt:P1813 ?short_name. }
    OPTIONAL { ?series wdt:P571 ?beginnings. }
    OPTIONAL { ?series wdt:P5127 ?WikiCFP_identifier. }
    OPTIONAL { ?series wdt:P8926 ?DBLP_identifier. }
    }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
    }
}''')

In [11]:
# untangling the results in order to get a dataframe

results = temp['results']['bindings']

conf_labels = np.array([result['conferencesLabel']['value'] for result in results])
series_labels = np.array([result['seriesLabel']['value'] if 'seriesLabel' in result.keys() else pd.NA for result in results])
short_name = np.array([result['short_name']['value'] if 'short_name' in result.keys() else pd.NA for result in results])
beginnings = np.array([result['beginnings']['value'] if 'beginnings' in result.keys() else pd.NA for result in results])
wikicfp_identifier = np.array([result['WikiCFP_identifier']['value'] if 'WikiCFP_identifier' in result.keys() else pd.NA for result in results])
dblp_identifier = np.array([result['DBLP_identifier']['value'] if 'DBLP_identifier' in result.keys() else pd.NA for result in results])

In [12]:
# zip all arrays together to create a DataFrame

together = zip(conf_labels, series_labels, short_name, beginnings, wikicfp_identifier, dblp_identifier)

df = pd.DataFrame(together, columns=['conf_labels', 'series_labels', 'short_name', 'beginnings', 'WikiCFP_identifier', 'DBLP_identifier'])
df.sample(200, random_state=42)

,conf_labels,series_labels,short_name,beginnings,WikiCFP_identifier,DBLP_identifier
4575,2006 ACM/IEEE Symposium on Architecture for Ne...,Symposium on Architectures for Networking and ...,ANCS,<NA>,171,conf/ancs
1488,"Management of Multimedia on the Internet, 5th ...",IFIP/IEEE International Conference on Manageme...,MMNS,1997-01-01T00:00:00Z,<NA>,conf/mmns
4970,"ICEIS Doctoral Consortium, 4th ICEIS Doctoral ...",ICEIS Doctoral Consortium,DCEIS,<NA>,<NA>,conf/dceis
8009,Digitizing Cultural Heritage and its Innovativ...,<NA>,<NA>,<NA>,<NA>,<NA>
3027,"Digital Enterprise Computing (DEC 2016), Böbli...",Digital Enterprise Computing,DEC,<NA>,<NA>,conf/dec
...,...,...,...,...,...,...
6916,3rd ACM Conference on Recommender Systems,The ACM Conference Series on Recommender Systems,RecSys,2007-01-01T00:00:00Z,<NA>,conf/recsys
5993,22nd International Symposium on High-Performan...,IEEE International Symposium on High-Performan...,HPDC,1992-01-01T00:00:00Z,1226,conf/hpdc
5664,17th International ACM SIGACCESS Conference on...,International ACM SIGACCESS Conference on Comp...,ASSETS,1994-01-01T00:00:00Z,243,conf/assets
718,"Information and Communication Security, Second...",International Conference on Information and Co...,ICICS,1997-01-01T00:00:00Z,1385,conf/icics


In [13]:
print(df.shape)
df.isna().sum()

(8910, 6)


conf_labels              0
series_labels         1039
short_name            1557
beginnings            6363
WikiCFP_identifier    5238
DBLP_identifier       1540
dtype: int64

# Let's check an example and its possible queries for the different databases:

example: conference series IWCMC

In [14]:
df.loc[df['short_name'] == 'IWCMC']

,conf_labels,series_labels,short_name,beginnings,WikiCFP_identifier,DBLP_identifier
907,7th International Wireless Communications and ...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
5127,2016 International Wireless Communications and...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
5128,International Wireless Communications and Mobi...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
5130,International Conference on Wireless Communica...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
5131,6th International Wireless Communications and ...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
5132,International Conference on Wireless Communica...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc
7248,International Conference on Wireless Communica...,International Conference on Wireless Communica...,IWCMC,<NA>,1824,conf/iwcmc


## In Wikidata:
events for the years: 2007, 2009, 2010, 2011, 2014, 2016 and 2022
--> I also could not find other entries that are just not connected to the series...
_________________________________________________________________________________________________________

## In WikiCFP:
events for the years: <b>2008</b>, 2009, 2010, <b>2012</b>, <b>2013</b>, 2016 and <b>2017</b>
http://www.wikicfp.com/cfp/program?id=1824&s=IWCMC&f=International%20Conference%20on%20Wireless%20Communications%20and%20Mobile%20Computing

## In DBLP:
events for the years: <b>2006</b>, 2007, 2009, 2010, 2011, <b>2012</b>, <b>2013</b>, 2014, <b>2015</b>, 2016, <b>2017</b>, <b>2018</b>, <b>2019</b>, <b>2020</b>, <b>2021</b>, 2022, <b>2023</b>
https://dblp.org/db/conf/iwcmc/index.html
every series is well organized

## In Crossref:
https://search.crossref.org/search/works?q=iwcmc&from_ui=yes
There is a lot of unorganized information about this conference series. There is no such thing as the overall conference series with an enumeration of all events. Structuring could be a lot of work.

## In CORE:
https://core.ac.uk/search?q=iwcmc
There only seems to be information about papers. Maybe we can use the papers property in order to deduce information about conference events.

## In BASE:
https://www.base-search.net/Search/Results?type=all&lookfor=IWCMC&ling=1&oaboost=1&keepFilters=1&filter%5B%5D=f_dctypenorm%3A%2213%22&name=&thes=&refid=dcresde&newsearch=1
Information seems to be a bit more organized than in Crossref. All information seems to be imported from Crossref though.
The reference to the events are directed to IEEE. Here we also get a list of all conference events:

## In IEEE:
https://ieeexplore.ieee.org/xpl/conhome/1002410/all-proceedings
events for the years: <b>2008</b>, 2011, <b>2012</b>, <b>2013</b>, 2014, <b>2015</b>, 2016, <b>2017</b>, <b>2018</b>, <b>2019</b>, <b>2020</b>, <b>2021</b>, 2022 and <b>2023</b>

# Also there are duplicates: Should we merge those?

In [54]:
groupedby_dblp_df = df.groupby('DBLP_identifier').agg(set)
groupedby_dblp_df

,conf_labels,series_labels,short_name,beginnings,WikiCFP_identifier
DBLP_identifier,,,,,
conf/3dica,"{Leveraging the Semantics of Topic Maps, Secon...","{Three-Dimensional Image Processing, Measureme...",{3DIPM},{1998-01-01T00:00:00Z},{<NA>}
conf/3dim,"{Fourth International Conference on 3D Vision,...",{International Conference on 3D Vision},{3DV},{<NA>},{<NA>}
conf/3dui,"{3DUI 2006, 3DUI 2016, 3DUI 2019, 3DUI 2018, 3...",{IEEE Symposium on 3D User Interfaces},{3DUI},{<NA>},{<NA>}
conf/5gwf,"{2nd IEEE 5G World Forum, 5GWF 2019, Dresden, ...",{5G World Forum},{5GWF},{2018-01-01T00:00:00Z},{<NA>}
conf/5gwn,{5G for Future Wireless Networks - 2nd EAI Int...,{International Conference on 5G for Future Wir...,{5GWN},{2017-01-01T00:00:00Z},{<NA>}
...,...,...,...,...,...
conf/www,{Sixth International World Wide Web Conference...,{The Web Conference},{WWW},{1994-01-01T00:00:00Z},{3182}
conf/xps,"{Expertensysteme 93, 2. Deutsche Tagung Expert...",{German Conference on Knowledge-Based Systems},{XPS},{<NA>},{<NA>}
conf/xpu,{Agile Processes in Software Engineering and E...,{International Conference on Agile Software De...,{XP},{<NA>},{3186}


In [55]:
groupedby_dblp_df = groupedby_dblp_df.assign(length = groupedby_dblp_df['series_labels'].apply(len))

In [56]:
groupedby_dblp_df.loc[groupedby_dblp_df['length'] > 1]

,conf_labels,series_labels,short_name,beginnings,WikiCFP_identifier,length
DBLP_identifier,,,,,,
conf/ecml,{Machine Learning and Knowledge Discovery in D...,{European Conference on Machine Learning and P...,"{ECML, ECML PKDD}",{<NA>},{<NA>},2


In [57]:
groupedby_shortname_df = df.groupby('short_name').agg(set)
groupedby_shortname_df

,conf_labels,series_labels,beginnings,WikiCFP_identifier,DBLP_identifier
short_name,,,,,
*SEM,{The Eighth Joint Conference on Lexical and Co...,{Joint Conference on Lexical and Computational...,{<NA>},{<NA>},{conf/starsem}
3DIPM,"{Leveraging the Semantics of Topic Maps, Secon...","{Three-Dimensional Image Processing, Measureme...",{1998-01-01T00:00:00Z},{<NA>},{conf/3dica}
3DUI,"{3DUI 2006, 3DUI 2016, 3DUI 2019, 3DUI 2018, 3...",{IEEE Symposium on 3D User Interfaces},{<NA>},{<NA>},{conf/3dui}
3DV,"{Fourth International Conference on 3D Vision,...",{International Conference on 3D Vision},{<NA>},{<NA>},{conf/3dim}
3GSE,"{28th USENIX Security Symposium, USENIX Securi...",{USENIX Security Symposium},{<NA>},{<NA>},{conf/uss}
...,...,...,...,...,...
iNetSeC,{Geospatial Free and Open Source Software in t...,{Open Research Problems in Network Security},{<NA>},{<NA>},{conf/ifip11-4}
iTAG,{2014 International Conference on Interactive ...,{International Conference on Interactive Techn...,{<NA>},{<NA>},{conf/itag}
icSPORTS,{3rd International Congress on Sport Sciences ...,{International Congress on Sport Sciences Rese...,{<NA>},{<NA>},{conf/icsports}


In [58]:
groupedby_shortname_df = groupedby_shortname_df.assign(length = groupedby_shortname_df['series_labels'].apply(len))

In [59]:
groupedby_shortname_df.loc[groupedby_shortname_df['length'] > 1]

,conf_labels,series_labels,beginnings,WikiCFP_identifier,DBLP_identifier,length
short_name,,,,,,
ACE,{International Conference on Advances in Compu...,"{Australasian Computing Education Conference, ...","{2004-01-01T00:00:00Z, <NA>}","{24, 23}","{conf/ACMace, conf/ace}",2
ACL,"{ACL 2010, 56th Annual Meeting of the Associat...",{Proceedings of the conference. Association fo...,{<NA>},"{37, <NA>}","{conf/acl, <NA>}",2
ACSAC,{ACSAC '20: Annual Computer Security Applicati...,{Annual Computer Security Applications Confere...,{<NA>},"{46, 45}","{conf/acsac, conf/aPcsac}",2
AH,"{User Modeling, Adaption and Personalization -...","{Augmented Human International Conference, Int...",{<NA>},"{90, 89}","{conf/aughuman, conf/ah}",2
AIM,{2018 IEEE/ASME International Conference on Ad...,{IEEE International Conference on Advanced Int...,{<NA>},{<NA>},"{conf/aim, conf/aimech}",2
AIPR,{International Conference on Artificial Intell...,{Artificial Intelligence and Pattern Recogniti...,{<NA>},"{117, <NA>}","{conf/aipr2, conf/aiprf}",2
BDA,{Big Data Analytics - 5th International Confer...,{International Conference on Big Data Analytic...,"{1985-01-01T00:00:00Z, <NA>}",{<NA>},"{conf/bigda, conf/bda}",2
CCS,{2019 ACM SIGSAC Conference on Computer and Co...,"{Circuits, Signals, and Systems, Conference on...","{1993-01-01T00:00:00Z, <NA>}","{399, <NA>}","{conf/iastedCCS, conf/ccs}",2
CIC,{Eighth Color Imaging Conference: Color Scienc...,"{Color Imaging Conference, International Confe...","{2015-01-01T00:00:00Z, <NA>}","{449, <NA>}","{conf/coinco, conf/imaging}",2
